In [ ]:
from pathlib import Path
import pandas as pd
from pgscen.command_line import (
    load_wind_data, split_actuals_hist_future, split_forecasts_hist_future)


cur_path = Path("day-ahead_wind_scenario.ipynb").parent.resolve()
data_dir = Path(cur_path, '..', "data").resolve()
wind_site_actual_df, wind_site_forecast_df, wind_meta_df = load_wind_data()

scenario_start_time = pd.to_datetime('2018-07-01 06:00:00',utc=True)
scen_timesteps = pd.date_range(start=scenario_start_time,
                               periods=24, freq='H')

(wind_site_actual_hists,
     wind_site_actual_futures) = split_actuals_hist_future(
            wind_site_actual_df, scen_timesteps, in_sample=True)
(wind_site_forecast_hists,
     wind_site_forecast_futures) = split_forecasts_hist_future(
            wind_site_forecast_df, scen_timesteps, in_sample=True)

In [ ]:
from pgscen.engine import GeminiEngine

scenario_count=1000

ge = GeminiEngine(wind_site_actual_hists, wind_site_forecast_hists,
                  scenario_start_time, wind_meta_df, 'wind')

dist = ge.asset_distance().values
ge.fit(dist / (10 * dist.max()), 5e-2)

ge.create_scenario(scenario_count, wind_site_forecast_futures)

In [ ]:
ge.model.hist_dev_df

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [19, 11]

plt_asset = ge.asset_list[21]
plt_ts = ge.scen_timesteps[18]
print("{} at {}".format(plt_asset, plt_ts))
_ = plt.hist(ge.model.scen_df[plt_asset][plt_ts], bins=30)

In [ ]:
out_dir = Path(data_dir, '..')
ge.write_to_csv(out_dir, wind_site_actual_futures, write_forecasts=True)
df = pd.read_csv("{}/20180701/wind/Wilson_Ranch.csv".format(out_dir))
print(df)

In [ ]:
import numpy as np

idx = np.random.randint(1000)
plt.plot(df.iloc[0, 2:],label='actual')
plt.plot(df.iloc[1, 2:],label='forecast')
plt.plot(df.iloc[idx, 2:],label='scenario')
_ = plt.legend()

In [ ]:
_ = plt.hist(df.iloc[2:, :]['0600'],bins=30)